# 🛠️ **Instalação de Dependências**
Instala as bibliotecas necessárias:
- **cloudscraper**: para contornar bloqueios HTTP comuns.
- **playwright**: para automação e navegação web.
- **sofascore_wrapper**: interface simplificada com a API do Sofascore.

In [ ]:
!pip install cloudscraper playwright sofascore_wrapper
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 3.2 MB/s eta 0:00:00
171.6 MiB [] 0% 130.6s171.6 MiB [] 0% 54.0s171.6 MiB [] 0% 25.8s171.6 MiB [] 0% 16.5s171.6 MiB [] 0% 8.4s171.6 MiB [] 1% 5.2s171.6 MiB [] 2% 4.3s171.6 MiB [] 3% 3.4s171.6 MiB [] 4% 2.9s171.6 MiB [] 5% 2.7s171.6 MiB [] 5% 2.9s171.6 MiB [] 6% 2.8s171.6 MiB [] 6% 2.7s171.6 MiB [] 7% 2.5s171.6 MiB [] 8% 2.4s171.6 MiB [] 9% 2.3s171.6 MiB [] 10% 2.2s171.6 MiB [] 11% 2.1s171.6 MiB [] 13% 2.0s171.6 MiB [] 14% 1.9s171.6 MiB [] 15% 1.9s171.6 MiB [] 16% 1.8s171.6 MiB [] 17% 1.7s171.6 MiB [] 19% 1.6s171.6 MiB [] 20% 1.6s171.6 MiB [] 21% 1.5s171.6 MiB [] 23% 1.4s171.6 MiB [] 24% 1.4s171.6 MiB [] 25% 1.4s171.6 MiB [] 26% 1.4s171.6 MiB [] 27% 1.4s171.6 MiB [] 28% 1.3s171.6 MiB [] 30% 1.3s171.6 MiB [] 32% 1.2s171.6 MiB [] 33% 1.2s171.6 MiB [] 35% 1.1s171.6 MiB [] 36% 1.1

# ⚽️ **Mapeamento dos Campeonatos**
Define um dicionário com os IDs dos torneios e temporadas usados na API Sofascore.
- **Nota**: Ajuste conforme a necessidade adicionando/removendo campeonatos.
Funções Auxiliares

## 🔧 **Funções Auxiliares**

### 🧹 `extract_float`
- Converte valores percentuais (strings) em valores numéricos do tipo float.

### 🎯 `position_map`
- Define mapeamento simplificado para as posições dos jogadores.

### 📊 `position_required_stats`
- Lista estatísticas essenciais por posição do jogador (para validação dos dados).

### 📈 `classify_rating`
- Classifica numericamente o desempenho dos jogadores em categorias ("bom", "mediano", "ruim") com base na nota fornecida pelo Sofascore.

## 📝 **Processamento de Dados dos Jogadores**

### ⚙️ `process_player_data`
- Extrai estatísticas relevantes dos jogadores, filtrando aqueles com estatísticas mínimas.
- Retorna lista organizada de dicionários com atributos importantes.

## 📚 **Criação do Dataset para Fine-tuning**

### 🧪 `create_training_data`
- Formata as estatísticas dos jogadores em textos naturais.
- Cria exemplos de treinamento no formato padrão para fine-tuning (estilo Alpaca).
- Retorna lista com instruções, entradas e respostas categorizadas.

## 🌐 **Extração de Eventos e Lineups da API Sofascore**

### 📅 `fetch_events_for_champ`
- Recupera eventos (partidas) para cada campeonato e temporada especificados.
- Interage com a API do Sofascore paginadamente.

### 📑 `fetch_event_details`
- Obtém detalhes da partida e escalações dos jogadores para cada evento.
- Trata exceções HTTP (`404`) que indicam indisponibilidade de dados.

## 🚀 **Execução Principal do Script**

### 🔍 `main`
- Itera sobre os campeonatos definidos, obtendo eventos, detalhes e lineups.
- Processa e salva os dados coletados em arquivos CSV individuais por campeonato.
- Consolida todos os dados em um arquivo único (`processed_player_data_multi_championship.csv`).
- Gera dataset de treinamento (`player_performance_analysis_dataset.csv`) com contexto adicional para simular recuperação semântica (RAFT).

## ⏳ **Execução Assíncrona no Google Colab**

- Aplica o `nest_asyncio` para permitir a execução assíncrona dentro do ambiente do Colab.
- Executa o loop principal com asyncio.

In [ ]:
import os
import json
import asyncio
from collections import defaultdict

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

from sofascore_wrapper.api import SofascoreAPI  # ajuste conforme o nome real do pacote

# Pasta de saída (pode ser um diretório montado do Google Drive)
output_folder = os.getenv('OUTPUT_FOLDER', '/content/drive/My Drive/dados_rag_new')
os.makedirs(output_folder, exist_ok=True)

# Mapeamento de campeonatos para códigos de API
championships = {
    # "Brasileirão Betano": {"tournament": 325, "season": 58766},
    # "Premier League":   {"tournament": 17,  "season": 61627},
    # "La Liga":          {"tournament": 8,   "season": 61643},
    # "Bundesliga":       {"tournament": 35,  "season": 63516},
    # "Serie A":          {"tournament": 23,  "season": 63515},
    # "Ligue 1":          {"tournament": 34,  "season": 61736},
    # "Liga Portugal":    {"tournament": 238, "season": 63670},
    # "Liga Profesional de Fútbol": {"tournament": 155, "season": 70268},
    "Eredivisie":       {"tournament": 37,  "season": 61666}
}

# Converte string percentual para float
def extract_float(val):
    s = str(val).replace('%','').replace(',','.')
    try:
        return float(s)
    except:
        return 0.0

# Mapeamento de posições e estatísticas necessárias
position_map = {'G':'goalkeeper','D':'defender','M':'midfield','F':'forward'}
position_required_stats = {
    'forward': ['goals','goalAssist','onTargetScoringAttempt'],
    'midfield': ['accuratePass','keyPass','goalAssist'],
    'defender': ['totalTackle','totalClearance','interceptionWon'],
    'goalkeeper': ['saves','goalsPrevented','goodHighClaim']
}

# Classifica rating em bom/mediano/ruim
def classify_rating(rating):
    try:
        rating = float(rating)
    except:
        return "desconhecido"
    if rating >= 7.5:
        return "bom"
    if rating >= 6.0:
        return "mediano"
    return "ruim"

# Processa lista de jogadores extraindo features
def process_player_data(players, event_description):
    processed = []
    keys = ['goals','goalAssist','onTargetScoringAttempt','accuratePass',
            'keyPass','totalTackle','totalClearance','interceptionWon',
            'saves','goalsPrevented','goodHighClaim','rating']
    for p in players:
        pos = position_map.get(p.get('position'),'Unknown')
        if pos == 'Unknown':
            continue
        stats = p.get('statistics',{})
        # verifica estatísticas mínimas
        missing = sum(1 for stat in position_required_stats[pos] if stats.get(stat,0) == 0)
        if missing > 1:
            continue
        label = classify_rating(stats.get('rating',0))
        feat = {
            'player_name': p['player']['name'],
            'position': pos,
            'rating': stats.get('rating',0),
            'ratingLabel': label,
            'event_id': p.get('event_id'),
            'event_description': event_description,
            'championship': p.get('championship','desconhecido')
        }
        for k in keys:
            feat[k] = stats.get(k,0)
        processed.append(feat)
    return processed

# Cria exemplos de treino
def create_training_data(df: pd.DataFrame) -> list:
    examples = []
    for _, r in df.iterrows():
        inp = (
            f"Na partida {r['event_description']}, o jogador {r['player_name']}, atuando como {r['position']}, "
            f"marcou {r['goals']} gols, deu {r['goalAssist']} assistências, fez {r['onTargetScoringAttempt']} chutes a gol, "
            f"{r['accuratePass']} passes precisos, {r['keyPass']} chances, {r['totalTackle']} desarmes, {r['totalClearance']} clearances, "
            f"{r['interceptionWon']} interceptações, {r['saves']} defesas, preveniu {r['goalsPrevented']} gols, {r['goodHighClaim']} high claims."
        )
        q = (
            f"Considerando os dados, como avaliaria o desempenho do jogador {r['player_name']} "
            f"na partida {r['event_description']}? Responda bom, mediano ou ruim."
        )
        inst = "Classifique o desempenho do jogador como bom, mediano ou ruim."
        examples.append({
            'instructions': inst,
            'input': inp,
            'question': q,
            'response': r['ratingLabel'],
            'match': r['event_description'],
            'championship': r['championship'],
            'player': r['player_name'],
            'position': r['position']
        })
    return examples

# Busca eventos de um único campeonato
async def fetch_events_for_champ(api: SofascoreAPI, tournament:int, season:int):
    events = []
    idx = 0
    while True:
        ep = f"/unique-tournament/{tournament}/season/{season}/events/last/{idx}"
        try:
            data = await api._get(ep)
        except Exception as e:
            if '404' in str(e):
                break
            raise
        if not data.get('events'):
            break
        events.extend(data['events'])
        idx += 1
        await asyncio.sleep(0.1)
    return events

# Busca detalhes e lineups de um evento, ignorando 404 de lineups
async def fetch_event_details(api: SofascoreAPI, event_id:int):
    try:
        detail = await api._get(f"/event/{event_id}")
    except Exception as e:
        if '404' in str(e):
            return None, None
        raise
    await asyncio.sleep(0.1)
    try:
        lineups = await api._get(f"/event/{event_id}/lineups")
    except Exception as e:
        if '404' in str(e):
            return detail, {}
        raise
    await asyncio.sleep(0.1)
    return detail, lineups

async def main():
    api = SofascoreAPI()
    all_players = []
    try:
        # Itera por campeonato
        for champ, codes in championships.items():
            print(f"Processando {champ}...")
            evs = await fetch_events_for_champ(api, codes['tournament'], codes['season'])
            print(f"{champ}: {len(evs)} eventos coletados")
            players = []
            for ev in evs:
                detail, lineups = await fetch_event_details(api, ev['id'])
                if detail is None:
                    continue
                status = detail.get('event', {}).get('status', {}).get('description')
                if status != 'Ended':
                    continue
                desc = f"{detail['event']['homeTeam']['name']} vs {detail['event']['awayTeam']['name']} do campeonato {champ}"
                for side in ('home', 'away'):
                    for p in lineups.get(side, {}).get('players', []):
                        p['event_id'] = ev['id']
                        p['championship'] = champ
                    players.extend(process_player_data(lineups.get(side, {}).get('players', []), desc))
            # Salva DF deste campeonato
            df_ch = pd.DataFrame(players)
            path = os.path.join(output_folder, f"players_{champ.replace(' ', '_')}.csv")
            df_ch.to_csv(path, index=False)
            print(f"Dados de {champ} salvos em: {path}")
            all_players.extend(players)
        # Salva todos juntos
        df_all = pd.DataFrame(all_players)
        all_path = os.path.join(output_folder, 'processed_player_data_multi_championship.csv')
        df_all.to_csv(all_path, index=False)
        print(f"Todos os dados salvos em: {all_path}")
        # Dataset de treino
        train = create_training_data(df_all)
        df_train = pd.DataFrame(train)
        df_train['rag_input'] = df_train.apply(
            lambda r: r['input'] + "\n---\n" + "\n---\n".join(
                df_train.drop(r.name)['input'].sample(2).tolist()
            ), axis=1
        )
        train_path = os.path.join(output_folder, 'player_performance_analysis_dataset.csv')
        df_train.to_csv(train_path, sep=';', index=False)
        print(f"Dataset de treinamento salvo em: {train_path}")
    finally:
        await api.close()

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Processando Eredivisie...
Eredivisie: 325 eventos coletados
Dados de Eredivisie salvos em: /content/drive/My Drive/dados_rag_new/players_Eredivisie.csv
Todos os dados salvos em: /content/drive/My Drive/dados_rag_new/processed_player_data_multi_championship.csv
Dataset de treinamento salvo em: /content/drive/My Drive/dados_rag_new/player_performance_analysis_dataset.csv
